# Getting started with heating in impulsePy

This document covers all the things you need to know to get started with impulsePy. To run this script, start an Impulse experiment with temperature control enabled. 

You can set Impulse to stub mode so that you can run your script without the need of any hardware. For instructions on how to do this, please contact me at merijn.pen@denssolutions.com.

For this introduction, the following libraries are required:

In [ ]:
import impulsePy as impulse # Required for the communication with Impulse
from IPython.display import display # Used for this guide to display the dataFrames in a nice way

## Chapter 1. Subscribing to channels & receiving data

### 1.1 Subscribing to channels

To receive data from impulse for a certain stimulus, we can subscribe to that stimulus using `impulse.<stimulus>.data.subscribe()`. 

In [ ]:
# Connect to the Impulse channels
impulse.heat.data.subscribe()

### 1.2 Getting a datapoint

Now that we are subsribed to the data and event channel, we can read the last measurements for a stimulus by using 
`impulse.<stimulus>.data.getLastData()`.

We can only receive data from Impulse when it is in "control" mode (when an experiment is running), so we can use `impulse.waitForControl()` to hold our script until Impulse is ready.

In this example we also pause the script using `impulse.sleep(sleep time in seconds)`.

In [ ]:
impulse.waitForControl() #Make sure that Impulse has an active experiment running first.

impulse.sleep(2) # Wait 2 seconds so that we have received data from Impulse

heatDatapoint = impulse.heat.data.getLastData() # Receive the last dataset as a Dict
display(heatDatapoint) # Print the dict to see the contents.

# Get individual values from the dict this way:
print('\nThe current temperature is', heatDatapoint["temperatureMeasured"], "degrees Celsius") 

The getLastData command returns the measurements from that stimulus for a single timestamp in the form of a dictionary.
As shown in the last line of the code, values from a dictionary can be retrieved by adding the parameter name after the dict name.

The getLastData command instantly returns the last measurement that was received from Impulse. 
This means that when send the request again shortly after, you will probably receive the same measurement twice.

If you want to make sure that you receive a <b>new</b> measurement, you can use `impulse.<stimulus>.data.getNewData()`.
For slow devices, this will mean that the script waits some time for the device to send a new measurement.

In [ ]:
print("Getting last data fast results in the same measurement twice:")
print(impulse.heat.data.getLastData()["sequenceNumber"])
print(impulse.heat.data.getLastData()["sequenceNumber"])
print("Using getNewData, we make sure to get a different measurement every time we ask:")
print(impulse.heat.data.getNewData()["sequenceNumber"])
print(impulse.heat.data.getNewData()["sequenceNumber"])

### 1.3 Getting a dataFrame

You can also get a dataset with multiple measurements by using `impulse.<stimulus>.data.getDataFrame()`.

The getDataFrame command can be used in a few different ways:
- __Without any arguments__, which will return all data for that stimulus that was collected from the start of the subscription.
- __With one number__, which will return all data from that row onward. E.g. getDataFrame(-5) will return the last 5 rows.
- __With two numbers__, which will return all rows between the first to the second number.
- __Using flags instead of numbers__, same as the previous two but with flag names

In [ ]:
last5Meas = impulse.heat.data.getDataFrame(-5) # Gets the last 5 measurements in a dataFrame 
display(last5Meas)

specificRows = impulse.heat.data.getDataFrame(3,6) # Get rows 3 to 6 as a dataFrame
display(specificRows)

Except for when you want to receive the last n rows of data, rownumbers are quite meaningless if you want to get data from a particular section of your experiment.
To make that easier, you can also flag your data with a name during the experiment, and request that data using this flag later.

Setting flags can be done with: `impulse.<stimulus>.data.setFlag("Flag string here")`.
Flags are stored in the last column of the dataFrame under "flags".

You can then use the .getDataFrame command using the flags to get that data as a dataFrame.

In [ ]:
# Experiment with 5 cycles and flag the data in each cycle with the cycle-number.
for i in range(5):
    print(f"Running cycle {i}...")
    impulse.heat.data.setFlag(f"Cycle {i}")
    impulse.sleep(5) # Some stimulus controls here

cycle3Data = impulse.heat.data.getDataFrame("Cycle 3","Cycle 4") # Get the data between the start of Cycle 3 and Cycle 4
display(cycle3Data)

## Chapter 2. Stimulus Control

Each stimulus has its own set of control commands. All of these commands can be found in the "All ImpulsePy Commands" document.

ImpulsePy offers the following controls for controlling heat:

`impulse.heat.set(setpoint)`
> This command lets you set the temperature to a setpoint as a direct step.
> - __setpoint__: Temperature (°C)

`impulse.heat.startRamp(target, rampSpeedType, rampSpeedValue)`
> This command lets you start a temperature ramp from the current temperature to the target temperature. The second argument can either be "rampTime" or "rampRate" which determines what the value in the 3rd argument represents.
> - __target__: Target temperature (°C)
> - __rampSpeedType__: This argument should be "rampTime" or "rampRate", this determines what the next argument represents
> - __rampSpeedValue__: This argument either sets the duration of the ramp (seconds), when rampTime is set in the previous argument, or the rampRate (°C/s) if rampRate is set in the previous argument.

`impulse.heat.stopRamp()`
> This command lets you stop a running temperature ramp. This command has no arguments.

The use of these commands is demonstrated in the code below.

In [ ]:
impulse.heat.set(400) # Set the heat to 400 degrees
impulse.sleep(3)
impulse.heat.startRamp(500,"rampTime",20) # Ramp from the current temperature to 500 degrees with a ramptime of 20 seconds
impulse.sleep(8)
impulse.heat.stopRamp() # Stop the temperature ramp
impulse.sleep(3)
impulse.heat.startRamp(600,"rampRate",10) # Ramp from the current temperature to 600 degrees with a ramprate of 10 degrees/second

It can be useful to know when a ramp has finished. To do this, we can check the __busy__ variable of each stimulus.
This variable is set to True when the stimulus is not ramping, and False when the stimulus is ramping and therefore cannot be controlled.
The use of the busy status is demonstrated in the code below.

In [ ]:
print(impulse.heat.busy) #Prints False
impulse.heat.startRamp(200,'rampTime',10)
print(impulse.heat.busy) #Prints True
impulse.heat.stopRamp()
print(impulse.heat.busy) #Prints False again

You can use this busy parameter to wait until a ramp or sweepcycle is finished in the following way:

In [ ]:
impulse.heat.startRamp(330,'rampTime',5)

while impulse.heat.busy:
    impulse.sleep(0.1)
    
impulse.heat.startRamp(210,'rampTime',2)

### 2.4 Profile controls
The following commands allow you to control the profile player.

`impulse.profile.load(path)`
> Loads a profile into the profile player.
> - __path__: needs to be an absolute path to the profile file. If your profile file is in the standard Impulse profiles folder, you can use `impulse.profilesPath+'profilename.extension'`.

`impulse.profile.control(action)`
> Controls the profile playback buttons.
> - __action__: "play", "pause", "stop"

`impulse.profile.getStatus()`
> Returns the status of the profile player: running, paused or stopped.

In [ ]:
impulse.profile.load(impulse.profilesPath+'myProfile.temp.prf') # Replace myProfile.temp.prf with the file you want to load
impulse.profile.control("play")
impulse.sleep(5)
impulse.profile.control("pause")
impulse.sleep(5)
impulse.profile.control("stop")

## Chapter 3. States and errors

The Impulse software can be in three different states: 
1. <b>select</b> (stimuli selection page) 
2. <b>setup</b> (stimuli setup page) 
3. <b>control</b> (running experiment)

The state that Impulse is in can be requested using the command:`impulse.getStatus()`

The API controls only work in the control state, sending control commands in any other state will result in an `[INVALID OPERATION]` response. A way to make sure that impulse is in control mode is by using the `impulse.waitForControl()` command as shown earlier.

If you send a command that includes setpoints that are out-of-range, Impulse will return an `[INVALID INPUT]` error. The error message will include the ranges for the setpoints that were out-of-range, as shown below:

In [ ]:
impulse.heat.startRamp(9999,"rampTime", 999999999) #Setpoints out of range result in an invalidInput error

## Chapter 4. Disconnecting from Impulse after the experiment

The following line of code tells Impulse to stop sending the script datapoints and events.

In [ ]:
# Disconnect from Impulse
impulse.disconnect()